In [1]:
# # ### bz of low on ram reinput the dataset
import pandas as pd
# clean_defog_dataset = pd.read_parquet('Data/clean/clean_defog_dataset.parquet')
# clean_tdcsfog_dataset = pd.read_parquet('Data/clean/clean_tdcsfog_dataset.parquet')

In [2]:
# clean_dataset = pd.concat([clean_tdcsfog_dataset,clean_defog_dataset])
# clean_dataset.shape[0] == clean_defog_dataset.shape[0]+clean_tdcsfog_dataset.shape[0]
# clean_dataset.to_parquet('Data/clean/clean_mix_dataset.parquet')

In [3]:
clean_dataset = pd.read_parquet('Data/clean/clean_mix_dataset.parquet')

In [4]:
def oversampling(clean_dataset):
    feature_col = ['AccV','AccML','AccAP']
    label_col = ['StartHesitation','Turn','Walking', 'All_zero']

    feature_dataset = clean_dataset[feature_col]
    label_dataset = clean_dataset[label_col]
    print(f"The Feature :{feature_dataset.shape}, \n"
          f"The label {label_dataset.shape}")
    
    print(f"Because of Four Classes are imbalanced. To get high accuracy, oversampling is used")
    from imblearn.over_sampling import SMOTE
    import numpy as np
    # Instantiate the MultiLabelUnderSampler
    over_sampler = SMOTE()

    # Undersample the dataset
    X_resampled, y_resampled = over_sampler.fit_resample(feature_dataset.to_numpy(), label_dataset.to_numpy())
    
    SMOTE_features_dataset = pd.DataFrame(X_resampled, columns=feature_dataset.columns)
    
    
    SMOTE_labels_dataset = pd.DataFrame(y_resampled, columns=label_dataset.columns)
    print(f"The over sampling label shape : {SMOTE_labels_dataset.shape}")
    
    def check_all_four_class_condition(df):
        print(f"Check all four check condiiton in {df}")
        a = df[df.StartHesitation == 1].shape[0]
        b = df[df.Turn == 1].shape[0]
        c = df[df.Walking ==1].shape[0]
        d = df[df.All_zero == 1].shape[0]
        print(f"Number of Start Hesitation : {a}, \n"
              f"Number of Turn : {b}, \n"  
              f"Number of Walking : {c}, \n"
              f"Number of All_zero : {d}")
        print("Is Number of All four class is equal to total sampling :",
             df.shape[0] == a + b + c + d)
        
    check_all_four_class_condition(SMOTE_labels_dataset)
    
    oversampling_dataset = pd.concat([SMOTE_features_dataset,SMOTE_labels_dataset], 
                                     ignore_index= False, sort=False, axis=1)
    print(f"The shape of oversampling dataset is : {oversampling_dataset.shape[0]}")
    print(f"The number of duplication in dataset : {oversampling_dataset.duplicated().sum()}")
    # Drop duplication
    oversampling_dataset.drop_duplicates(inplace=True)
    print(f"The shape of oversampling after remove duplication :{oversampling_dataset.shape}")
  
    return oversampling_dataset
    
    

In [5]:
oversampling_dataset = oversampling(clean_dataset)
oversampling.head()

The Feature :(19089710, 3), 
The label (19089710, 4)
Because of Four Classes are imbalanced. To get high accuracy, oversampling is used
The over sampling label shape : (64970492, 4)
Check all four check condiiton in           StartHesitation  Turn  Walking  All_zero
0                       0     0        0         1
1                       0     0        0         1
2                       0     0        0         1
3                       0     0        0         1
4                       0     0        0         1
...                   ...   ...      ...       ...
64970487                0     0        1         0
64970488                0     0        1         0
64970489                0     0        1         0
64970490                0     0        1         0
64970491                0     0        1         0

[64970492 rows x 4 columns]
Number of Start Hesitation : 16242623, 
Number of Turn : 16242623, 
Number of Walking : 16242623, 
Number of All_zero : 16242623
Is Number of A

: 

: 

In [ ]:
oversampling_dataset.to_parquet('Data/clean/oversampling_mix_dataset.parquet')

In [ ]:
def split(oversampling_dataset):
        
    # 60% Train Data, 20% Validation Data, 20% Test Data
    # 80% Set Data(60% rain Data, 20% Validation Data) , 20% Test Data
    
    from sklearn.model_selection import train_test_split
    import random 
    random_seed = 54
    set_data, test_data = train_test_split(oversampling_dataset, test_size=0.2, random_state=True)
    print(f"The set data shape : {set_data.shape}\n"
          f"The test data shape : {test_data.shape}\n"
          f"Is the dataset still in range : "
          f"{oversampling_dataset.shape[0] == set_data.shape[0] + test_data.shape[0]}")
    
    print(f"Again Search for duplicaiton : \n "
          f"Set Data :{set_data.duplicated().sum()} \n"
          f"Test Data :{test_data.duplicated().sum()}")
    
    check_all_four_class_condition(set_data)
    check_all_four_class_condition(test_data)
    print("All task are finish")
    return set_data, test_data

In [ ]:
set_data , test_data = oversampling_and_split(clean_dataset)

In [ ]:
set_data.to_parquet('Data/Process/set_mix_data.parquet')
test_data.to_parquet('Data/Process/test_mix_data.parquet')

In [ ]:
def preprocessing_the_dataset(df):

    import random 
    random_seed = 54
    
    feature_col = ['AccV','AccML','AccAP']
    label_col = ['StartHesitation','Turn','Walking', 'All_zero']

    # make feature and label
    feature_dataset = df[feature_col]
    label_dataset = df[label_col]
    
    
    print("Train test split begin")
    from sklearn.model_selection import train_test_split
    train_feature, valid_feature, train_label, valid_label = train_test_split(feature_dataset, label_dataset, test_size=0.2, random_state=True)
    
    train_feature = np.array(train_feature) 
    valid_feature = np.array(valid_feature)
    train_label  = np.array(train_label)
    valid_label = np.array(valid_label)
    print("All task are finish")
    
    return train_feature, valid_feature, train_label, valid_label
    

In [ ]:
def preprocessing_test_dataset(df):

    import random 
    random_seed = 54
    
    feature_col = ['AccV','AccML','AccAP']
    label_col = ['StartHesitation','Turn','Walking', 'All_zero']

    # make feature and label
    feature_dataset = df[feature_col]
    label_dataset = df[label_col]
    
    feature_dataset = np.array(feature_dataset) 
    label_dataset  = np.array(label_dataset)
    print("All task are finish")
    
    return feature_dataset, label_dataset